In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
jarv_csv="/home/akis-linardos/Datasets/OPTIMAM/jarv_info.csv"
stge_csv="/home/akis-linardos/Datasets/OPTIMAM/stge_info.csv"
inbreast_csv="/home/lidia-garrucho/datasets/INBREAST/INbreast_updated_cropped_breast.csv"
cmmd_csv='/home/akis-linardos/Datasets/CMMD/info.csv'
bcdr_d01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D01_dataset/dataset_info.csv"
bcdr_d02_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D02_dataset/dataset_info.csv"
bcdr_dn01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-DN01_dataset/dataset_info.csv"

b_d01_csv=pd.read_csv(bcdr_d01_csv)
b_d02_csv=pd.read_csv(bcdr_d02_csv)
b_dn01_csv=pd.read_csv(bcdr_dn01_csv)
j_csv=pd.read_csv(jarv_csv)
s_csv=pd.read_csv(stge_csv)
i_csv=pd.read_csv(inbreast_csv)
c_csv=pd.read_csv(cmmd_csv)

In [8]:
jarv = j_csv[['view','status']]
stge = s_csv[['view','status']]
inbr = i_csv[['view','BIRADS']]
bcd1 = b_d01_csv[['view', 'classification']]
bcd2 = b_d02_csv[['view', 'classification']]
cmmd = c_csv[['view','status']]

In [9]:
jarv.iloc[0]

view            MLO
status    Malignant
Name: 0, dtype: object

In [10]:
stge.iloc[0]

view          CC
status    Benign
Name: 0, dtype: object

In [11]:
inbr

,view,BIRADS
0,CC,1
1,CC,3
2,MLO,1
3,MLO,3
4,CC,5
...,...,...
405,MLO,1
406,MLO,2
407,MLO,2
408,CC,2


In [12]:
def view_stats(dataset_pd, label='status'):
    r_benign, r_malign, l_benign, l_malign = 0, 0, 0, 0
    for row in dataset_pd.iterrows():
        if (row[1][label]=='Normal' or row[1][label]=='Interval Cancer'):
            continue

        if (row[1]['view']=='MLO')  and (row[1][label]=='Malignant' or row[1][label]=='Malign'):
            l_malign += 1
        elif (row[1]['view']=='CC') and (row[1][label]=='Malignant' or row[1][label]=='Malign'):
            r_malign += 1
        elif (row[1]['view']=='MLO')  and (row[1][label]=='Benign' or row[1][label]=='Benign'):
            l_benign += 1
        elif (row[1]['view']=='CC') and (row[1][label]=='Benign' or row[1][label]=='Benign'):
            r_benign += 1
        else:
            print("ERROR One of the two not recognized: {} {}".format(row[1]['view'], row[1][label]))
            break
    ml = l_malign/(l_malign+r_malign)
    mr = r_malign/(l_malign+r_malign)
    bl = l_benign/(l_benign+r_benign)
    br = r_benign/(l_benign+r_benign)
    return [ml, mr, bl, br]

In [13]:
def view_stats_inbreast(dataset_pd, label='BIRADS'):
    r_benign, r_malign, l_benign, l_malign = 0, 0, 0, 0
    for row in dataset_pd.iterrows():
        if (row[1][label]=='1'):
            continue

        if (row[1]['view']=='MLO') and (row[1][label] == '4' or row[1][label]== '5' or row[1][label]=='6'):
            l_malign += 1
        elif (row[1]['view']=='CC') and (row[1][label] == '4' or row[1][label]== '5' or row[1][label]=='6'):
            r_malign += 1
        elif (row[1]['view']=='MLO') and (row[1][label]=='2' or row[1][label]=='3'):
            l_benign += 1
        elif (row[1]['view']=='CC') and (row[1][label]=='2' or row[1][label]=='3'):
            r_benign += 1
        else:
            print("ERROR One of the two not recognized: {} {}".format(row[1]['view'], row[1][label]))
            break
    ml = l_malign/(l_malign+r_malign)
    mr = r_malign/(l_malign+r_malign)
    bl = l_benign/(l_benign+r_benign)
    br = r_benign/(l_benign+r_benign)
    return [ml, mr, bl, br]

In [15]:
view_stats(stge)

ERROR One of the two not recognized: XCCM Malignant


[0.5023809523809524,
 0.4976190476190476,
 0.4975845410628019,
 0.5024154589371981]

In [17]:
bcdr = pd.concat([bcd1, bcd2])

In [21]:
view_DF_stats = pd.DataFrame(np.array([view_stats(stge), view_stats(jarv), view_stats_inbreast(inbr, label='BIRADS')
, view_stats(bcdr, 'classification'), view_stats(cmmd)]), columns=['MLO+Malignant', 'CC+Malignant', 'MLO+Benign', 'CC+Benign'], index=['OPTIMAM(jarv)', 'OPTIMAM(stge)', 'InBreast', 'BCDR', 'CMMD'])

ERROR One of the two not recognized: XCCM Malignant
ERROR One of the two not recognized: XCCL Malignant
ERROR One of the two not recognized: MLO 4a


In [22]:
view_DF_stats

,MLO+Malignant,CC+Malignant,MLO+Benign,CC+Benign
OPTIMAM(jarv),0.502381,0.497619,0.497585,0.502415
OPTIMAM(stge),0.506098,0.493902,0.503185,0.496815
InBreast,0.500000,0.500000,0.600000,0.400000
BCDR,0.514019,0.485981,0.513238,0.486762
CMMD,0.500000,0.500000,0.500000,0.500000
